# Figure 1, but for Proxima Centauri b.

Overview of the model set-up for the Proxima b case. The *MassFlux* simulation covers the whole sphere, while the *HighRes* simulation is shown by superimposing a high-resolution grid that covers only the substellar region. The graphic shows cloud condensate (white isosurfaces), surface temperature (shading), and free troposphere wind vectors (arrows), focusing on (a) the day side, with the *HighRes* model domain and the cloud condensate isosurface, and (b) the night side of the planet. The cloud condensate is shown using a threshold of $10^{-5}~kg~kg^{-1}$ of total cloud condensate (liquid water plus ice). This figure is available online as an interactive zoom and rotatable figure.

[Skip code and jump to the figure](#Show-the-figure)

----------------------------------

## Import necessary libraries

Standard library

In [2]:
import warnings
import sat_data_to_cubes as wrangler
import os

warnings.filterwarnings("ignore")

3D visualization and mesh analysis library

In [3]:
import pyvista as pv
from pyvista import examples

Scientific stack

In [4]:
import iris
import numpy as np
import math

In [5]:
from aeolus.const import init_const
from aeolus.plot import subplot_label_generator
from aeolus.plot.pv import grid_for_scalar_cube_sph, grid_for_vector_cubes_sph

Set white background for 3d figures.

In [6]:
pv.set_plot_theme("default")

Global definitions.

## Load data

In [7]:
const = init_const("earth")

In [8]:
datadir = "/Users/erose/cira_workspace/cira-vertical-cloud-products/hdf_practice/files/files_grib/"

In [9]:
'''
Notes for tomorrow: You have to load in grib data by specifying the variable_id of the connected variable. In the example below, I'm collecting
"Cloud Mixing Ratio" data and "Cloud Ice Mixing-Ratio" data. The corresponding lat/long values should already be included, but if not, I can grab easily.

wanted coord_system(): GeogCS(7160000.0)

Tomorrow we shoud attempt loading in corresponding lat/long data, and then moving forward to understanding how the plotting works with the related data.
Nonetheless, it looks like we can load in ~all~ HRRR data, by just using the sidestep of specifying the variable id. This is great news and means we can load in data
fairly easily, without having to convert it to other means. Woo hoo!

'''
print(datadir)
dset_conus_clouds = iris.load(os.getcwd() + "/data/CONUS_goes16_abi_cloud3d_simple_20220925200117.nc")
dset_conus_terrain = iris.load(os.getcwd() + "/data/CONUS_goes16_abi_cloud3d_simple_20220925200117.nc")
dset_ll = np.load(datadir + "cld_data/2022001_lat_long.npz", mmap_mode="r")
lat_proj = dset_ll["arr_0"][0]
lon_proj = dset_ll["arr_0"][1] - 360.0
lat_min = np.min(lat_proj); lat_max = np.max(lat_proj)
lon_min = np.min(lon_proj); lon_max = np.max(lon_proj)
finalLatProj = np.arange(lat_min, lat_max, np.abs(lat_min - lat_max)/1059.0)
finalLonProj = np.arange(lon_min, lon_max, np.abs(lon_min - lon_max)/1799.0)
print(finalLatProj[0],finalLatProj[-1]); print(np.shape(finalLatProj))
print(finalLonProj[0],finalLonProj[-1]); print(np.shape(finalLonProj))

/Users/erose/cira_workspace/cira-vertical-cloud-products/hdf_practice/files/files_grib/
21.13812255859375 52.58593016598381
(1059,)
-134.09547424316406 -60.95788300257675
(1799,)


## Create `pyvista` objects for a composite plot

Set cloud condensate isosurface threshold.

In [10]:
TERRAIN_ISOSURF = [0.00002]  # [kg kg-1]
CC_ISOSURF = [0.00003,0.00004,0.00005,0.00006]  # [kg kg-1]
RADIUS = float(const.radius.data)  # Planet radius [m]
RADIUS

6371200.0

Set viewpoints.

In [11]:
CAM_POS_LATS = [21, 53]
CAM_POS_LONS = [-135, -59]

# CAM_POS_LATS = [10, 20]
# CAM_POS_LONS = [30, 120]

### Global

Set common visualisation propeties.

In [12]:
# z_scale = 75.0
# z_offset = RADIUS * 1.005

z_scale = 100.0
z_offset = RADIUS * 1.01

Extract the total cloud condensate.

In [13]:
terrain_cube = dset_conus_terrain.extract("cloud3d")[0]
cloud_cube = dset_conus_clouds.extract("cloud3d")[0]
terrain_cube.data = wrangler.maskByTerrain(terrain_cube.data)
cloud_cube.data = wrangler.maskCloudTypeToCmr(cloud_cube.data)
lats = np.arange(lat_min, lat_max, np.abs(lat_min - lat_max)/1350.0)
lons = np.arange(lon_min, lon_max, np.abs(lon_min - lon_max)/3250.0)
lvls = [float(i) * 650 for i in range(51)]
lat_coord = iris.coords.DimCoord(lats, "latitude", units="degrees")
lon_coord = iris.coords.DimCoord(lons, "longitude", units="degrees")
lvl_coord = iris.coords.DimCoord(lvls, "altitude", units="km")

cloud_cube.add_dim_coord(lvl_coord,data_dim=0)
cloud_cube.add_dim_coord(lat_coord,data_dim=1)
cloud_cube.add_dim_coord(lon_coord,data_dim=2)

terrain_cube.add_dim_coord(lvl_coord,data_dim=0)
terrain_cube.add_dim_coord(lat_coord,data_dim=1)
terrain_cube.add_dim_coord(lon_coord,data_dim=2)

HDF5-DIAG: Error detected in HDF5 (1.14.0) thread 1:
  #000: H5A.c line 679 in H5Aopen_by_name(): unable to synchronously open attribute
    major: Attribute
    minor: Can't open object
  #001: H5A.c line 641 in H5A__open_by_name_api_common(): unable to open attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Can't open object
  #002: H5A.c line 464 in H5A__open_common(): unable to open attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Can't open object
  #003: H5VLcallback.c line 1138 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #004: H5VLcallback.c line 1105 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #005: H5VLnative_attr.c line 161 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #006: H5Aint.c line 658 in H5A__open_by_name(): unable 

Create a grid for the cloud condensate and extract a 3D contour (isosurface).

In [48]:
cmr_data = cloud_cube
print(cloud_cube.data[cloud_cube.data > 0])
global_qct_cntr = (
    grid_for_scalar_cube_sph(
        cmr_data, z_scale=z_scale, z_offset=z_offset, label="global_qct_grid", isGeo=True
    )
    .cell_data_to_point_data()
    .contour(isosurfaces=CC_ISOSURF,compute_scalars=True)
    # .contour(isosurfaces=CC_ISOSURF,scalars=cloud_cube.data)
)
print(global_qct_cntr)

[7.e-05 7.e-05 7.e-05 ... 6.e-05 6.e-05 6.e-05]
PolyData (0x157561060)
  N Cells:	72513457
  N Points:	36307127
  X Bounds:	-5.395e+06, 4.289e+06
  Y Bounds:	-9.463e+06, -2.839e+06
  Z Bounds:	2.328e+06, 7.611e+06
  N Arrays:	1



### HighRes

Set common visualisation propeties.

In [14]:
z_scale = 120.0
z_offset = RADIUS * 1.02

Create a grid from the 3D total cloud condensate field and terrain field.

In [15]:
lam_terrain = grid_for_scalar_cube_sph(
    terrain_cube,
    z_scale=z_scale,
    z_offset=z_offset,
    label="lam_qct_grid", isGeo=True
)
print(lam_terrain)
lam_clouds = grid_for_scalar_cube_sph(
    cloud_cube,
    z_scale=z_scale,
    z_offset=z_offset,
    label="lam_qct_grid", isGeo=True
)
print(lam_clouds)

StructuredGrid (0x15954c280)
  N Cells:	223762500
  N Points:	228389252
  X Bounds:	-6.776e+06, 4.731e+06
  Y Bounds:	-9.736e+06, -2.817e+06
  Z Bounds:	2.328e+06, 8.292e+06
  Dimensions:	1351, 3251, 52
  N Arrays:	1

StructuredGrid (0x15954c4c0)
  N Cells:	223762500
  N Points:	228389252
  X Bounds:	-6.776e+06, 4.731e+06
  Y Bounds:	-9.736e+06, -2.817e+06
  Z Bounds:	2.328e+06, 8.292e+06
  Dimensions:	1351, 3251, 52
  N Arrays:	1



Create an isosurface of the cloud condensate.

In [16]:
print(type(lam_terrain))
# iris.save(lam_terrain, savedir + "conus_terrain_cube.nc")
# iris.save(lam_clouds, savedir + "conus_clouds_cube.nc")
cntr_terrain = lam_terrain.cell_data_to_point_data(pass_cell_data=True).contour(isosurfaces=TERRAIN_ISOSURF,compute_scalars=True)
cntr_cloud = lam_clouds.cell_data_to_point_data(pass_cell_data=True).contour(isosurfaces=CC_ISOSURF,compute_scalars=True)


<class 'pyvista.core.pointset.StructuredGrid'>


Extract a mesh from the HighRes data.

In [17]:
lam_terrain = (
    grid_for_scalar_cube_sph(
        terrain_cube[:, ::7, ::7],
        z_scale=z_scale,
        z_offset=z_offset,
        label="lam_dom", isGeo=True
    )
    .extract_geometry()
    .extract_all_edges()
)
lam_cloud = (
    grid_for_scalar_cube_sph(
        cloud_cube[:, ::7, ::7],
        z_scale=z_scale,
        z_offset=z_offset,
        label="lam_dom", isGeo=True
    )
    .extract_geometry()
    .extract_all_edges()
)

2023-04-25 11:07:28.819 ( 324.669s) [           12D8E]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2023-04-25 11:07:28.988 ( 324.838s) [           12D8E]    vtkExtractEdges.cxx:551   INFO| Created 493212 edges
2023-04-25 11:07:29.372 ( 325.222s) [           12D8E]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2023-04-25 11:07:29.543 ( 325.393s) [           12D8E]    vtkExtractEdges.cxx:551   INFO| Created 493212 edges


## Associate Scalars with Cloud Mesh

In [18]:
conus_clouds = cntr_cloud
conus_terrain = cntr_terrain
conus_clouds["Cloud Type"] = np.copy(conus_clouds.cell_data.active_scalars)
conus_clouds["opacity"] = np.copy(conus_clouds["Cloud Type"]) * 10000
print(conus_clouds["Cloud Type"].shape)
print(conus_clouds.cell_data.get_array("Cloud Type"))

(27277858,)
[4.e-05 3.e-05 4.e-05 ... 6.e-05 0.e+00 6.e-05]


## Prepare data for plotting

Pack grids and plotting arguments into a single list.

In [19]:
annotations = {
    0.00003: "Water",
    0.00004: "Supercooled",
    0.00005: "Mixed",
    0.00006: "Ice",
}
'''
Colors: ['blue', 'cyan', 'paleturquoise', 'lightcyan', 'grey', 'peru']
'''
# savedir = os.getcwd() +"/data/"
# np.savez_compressed(savedir + "conus_terrain_cube.nc", conus_terrain)
# np.savez_compressed(savedir + "conus_clouds_cube.nc", conus_clouds)
# earth_mesh = pv.Sphere(radius=RADIUS)
earth_mesh = wrangler.getGlobeMesh()
VIS_CONTAINER = [
    # },
    # {
    #     # Global cloud condensate
    #     "mesh": global_qct_cntr,
    #     "kwargs": {
    #         "cmap": "jet",
    #         "n_colors": 4,
    #         "clim": [0.00003, 0.00006],
    #         "opacity": 0.5,
    #         "show_scalar_bar": False,
    #         "smooth_shading": True,
    #     },
    # },
    {
        # Global surface temperature
        "mesh": earth_mesh,
        "kwargs": {
            # "color": "cadetblue",
            "show_scalar_bar": False,
            "smooth_shading": True,
        },
    },
    {
        # HighRes cloud condensate
        "mesh": conus_terrain,
        "kwargs": {
            "color": "peru",
            "specular": 0.5,
            "ambient": 0.5,
            "smooth_shading": True,
        },
    },
    {
        # HighRes cloud condensate
        "mesh": conus_clouds,
        "kwargs": {
            "cmap": ["blue","cyan","paleturquoise","lightcyan"],
            "n_colors": 4,
            "scalars": conus_clouds["Cloud Type"],
            "clim": [0.00003,0.00006],
            "opacity": conus_clouds["opacity"],
            "show_scalar_bar": True,
            "scalar_bar_args": {
                "title": "Cloud Type",
                "title_font_size": 23,
                "label_font_size": 15,
                "below_label": "Terrain",
                },
            "below_color": "peru",
            "specular": 0.5,
            "ambient": 0.5,
            "smooth_shading": True,
            "annotations": annotations,
        },
    },
    # {
    #     # HighRes domain edges
    #     "mesh": lam_terrain,
    #     "kwargs": {
    #         "style": "wireframe",
    #         "color": "k",
    #         "opacity": 0.2,
    #         "smooth_shading": True,
    #     },
    # },
    # {
    #     # HighRes domain edges
    #     "mesh": lam_cloud,
    #     "kwargs": {
    #         "style": "wireframe",
    #         "color": "k",
    #         "opacity": 0.2,
    #         "smooth_shading": True,
    #     },
    # },
]

## Plot the results

Assemble the plot.

In [22]:
iletters = subplot_label_generator()
# pv.global_theme.interactive = True
pv.set_jupyter_backend("ipyvtklink")
# p = gv.GeoPlotter(shape=(1, len(CAM_POS_LONS)), window_size=np.array([1024, 768 // 2]) * 2)
p = pv.Plotter(shape=(1, len(CAM_POS_LONS)), window_size=np.array([1024, 768 // 2]) * 2)

# with stars
# p.add_base_layer(texture=gv.natural_earth_hypsometric())
# p.add_coastlines(resolution="10m")
last_mesh = len(VIS_CONTAINER) - 1
for idx, (cam_lon, cam_lat) in enumerate(zip(CAM_POS_LONS, CAM_POS_LATS)):
    p.subplot(0, idx)
    p.add_background_image(examples.download_stars_jpg(),as_global=False)
    # p.add_background_image('stars.jpeg')

    p.add_text(f"({next(iletters)})", font="times", font_size=24)
    # curr_mesh = 0
    for plot_dict in VIS_CONTAINER:
        # print(plot_dict)
        p.add_mesh(plot_dict["mesh"], **plot_dict["kwargs"])
        # if curr_mesh == last_mesh:
        #     p.add_legend()
        # curr_mesh += 1
    p.set_position(pv.grid_from_sph_coords([cam_lon], [90 - cam_lat], [4.5e7]).points)
    # p.enable_point_picking()
    p.set_focus((0, 0, 0))
    p.set_viewup((0, 0, 1))

## Show the figure

In [23]:
# p.show()
p.show(jupyter_backend="ipyvtklink",return_viewer=True)#jupyter_backend="static")# jupyter_backend="ipyvtklink",return_viewer=True
# p.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=2048)